#students in that discipline : # faculty in the discipline
student data: https://www.wellesley.edu/oir/factbook/fall-enrollment-detail

! pip install google-cloud
! pip install google-cloud-storage
! pip install google-cloud-vision

In [72]:
import os
import re
import json
import pandas as pd
from google.cloud import vision
from google.cloud import storage
from google.protobuf import json_format
# Supported mime_types are: 'application/pdf' and 'image/tiff'
mime_type = 'application/pdf'


In [86]:
# How many pages should be grouped into each json output file.
batch_size = 2

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\abguh\Desktop\wdc\student-faculty-ratio-542887f8ebad.json"
client = vision.ImageAnnotatorClient()

In [87]:
feature = vision.Feature(
    type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

gcs_source = vision.GcsSource(uri='gs://bucket_777/Fall enrollment by major.pdf')
gcs_source = vision.GcsSource(uri=gcs_source_uri)
input_config = vision.InputConfig(
    gcs_source=gcs_source, mime_type=mime_type)

In [88]:
gcs_destination_uri='gs://bucket_777/enrollment_major_data '
gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)
output_config = vision.OutputConfig(
    gcs_destination=gcs_destination, batch_size=batch_size)


In [89]:
async_request = vision.AsyncAnnotateFileRequest(
    features=[feature], input_config=input_config,
    output_config=output_config)

operation = client.async_batch_annotate_files(
    requests=[async_request])


In [90]:
print('Waiting for the operation to finish.')
operation.result(timeout=180)

# Once the request has completed and the output has been
# written to GCS, we can list all the output files.
storage_client = storage.Client()


Waiting for the operation to finish.


In [91]:
match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
bucket_name = match.group(1)
prefix = match.group(2)

bucket = storage_client.get_bucket(bucket_name)


In [92]:
# List objects with the given prefix.
blob_list = list(bucket.list_blobs(prefix=prefix))
print('Output files:')
for blob in blob_list:
    print(blob.name)


Output files:
enrollment_major_data output-1-to-2.json


In [124]:
# Process the first output file from GCS.
# Since we specified batch_size=2, the first response contains
# the first two pages of the input file.
output = blob_list[0]

json_string = output.download_as_string()
response = json.loads(json_string)
print(responses[1]['fullTextAnnotation'].keys())

dict_keys(['pages', 'text'])


In [112]:
print(responses[0].keys())

dict_keys(['fullTextAnnotation', 'context'])


In [125]:
for i in range(2):
    annotation = responses[i]['fullTextAnnotation']['text']
    f = open('[Result txt file name].txt','a')
    f.write(annotation + '\n')
    f.close()
    
print("Done!")


Done!


In [102]:
first_page_response = response['responses'][0]
second_page_response = response['responses'][1]
annotation1 = first_page_response['fullTextAnnotation'] 
annotation2 = second_page_response['fullTextAnnnotation']
# Here we print the full text from the first page.
# The response contains more information:
# annotation/pages/blocks/paragraphs/words/symbols
# including confidence scores and bounding boxes
text = annotation1['text'] + annotation2['text']
print(text)

KeyError: 'fullTextAnnnotation'

In [85]:
columns = text[:49].split(" ")
majors = text[50:text.find('Liter..')].split(" ")
majors

['Grand',
 'Total',
 'Undeclared',
 'Economics',
 'Psychology',
 'Political',
 'Science',
 'Biological',
 'Sciences',
 'English',
 'Computer',
 'Science',
 'Neuroscience',
 'Mathematics',
 'International',
 'Relations',
 'Spanish',
 'History',
 'French',
 'Art',
 'History',
 'Chemistry',
 'Environmental',
 'Studies',
 "Women's",
 'Studies',
 'Media',
 'Arts',
 'and',
 'Sciences',
 'Biochemistry',
 '&',
 'Biological',
 '..',
 'Anthropology',
 'Philosophy',
 'Sociology',
 'American',
 'Studies',
 'Cognitive&Linguistic',
 'Scie..',
 'Peace',
 'and',
 'Justice',
 'Studies',
 'Cinema',
 'and',
 'Media',
 'Studies',
 'Physics',
 'Architecture',
 'English',
 'and',
 'Creative',
 'Writi..',
 'East',
 'Asian',
 'Studies',
 'Classical',
 'Civilization',
 'Geosciences',
 'Music',
 'Art',
 'Studio',
 'Religion',
 'Middle',
 'Eastern',
 'Studies',
 'Theatre',
 'Studies',
 'Japanese',
 'Language',
 '&',
 'Cul..',
 'Africana',
 'Studies',
 'Chinese',
 'Language',
 '&',
 'Cultu..',
 'Comparative',
 'L